In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# Load and preprocess data
def load_and_preprocess_data(filepath):
    loc1 = pd.read_csv(filepath)
    loc1['Time'] = pd.to_datetime(loc1['Time'])
    loc1['Year'] = loc1['Time'].dt.year
    loc1['Month'] = loc1['Time'].dt.month
    features = ['Year', 'Month', 'temperature_2m', 'relativehumidity_2m', 
                'dewpoint_2m', 'windspeed_10m', 'windspeed_100m', 
                'winddirection_10m', 'winddirection_100m', 'windgusts_10m']
    X = loc1[features]
    y = loc1['Power']
    return X, y

In [3]:
# Define parameter grid for RandomizedSearchCV
def get_param_grid():
    return {
        'n_estimators': [50, 100, 150],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    }

In [4]:
# Calculate Adjusted R²
def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

In [5]:
# Train and evaluate Random Forest model
def train_and_evaluate_rf(X, y):
    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, random_state=40)
    
    # Perform Randomized Search with Cross-Validation
    param_grid = get_param_grid()
    model = RandomForestRegressor(random_state=40)
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, 
                                       n_iter=min(10, len(param_grid['n_estimators']) * len(param_grid['max_depth']) *
                                                  len(param_grid['min_samples_split']) * len(param_grid['min_samples_leaf'])), 
                                       cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1, random_state=40)
    random_search.fit(X_train, y_train)
    
    # Best parameters and model
    best_model = random_search.best_estimator_
    print("Best parameters found: ", random_search.best_params_)

    # Evaluate on test set
    predictions = best_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    adj_r2 = adjusted_r2(r2, X_test.shape[0], X_test.shape[1])

    print(f'\nRandom Forest Model Evaluation:')
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')
    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'R² Score: {r2}')
    print(f'Adjusted R² Score: {adj_r2}')

    # Cross-validation scores on the best model
    cv_results_mse = cross_val_score(best_model, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
    mse_cv = -cv_results_mse.mean()
    rmse_cv = np.sqrt(mse_cv)
    cv_results_mae = cross_val_score(best_model, X_scaled, y, cv=5, scoring='neg_mean_absolute_error')
    mae_cv = -cv_results_mae.mean()
    cv_results_r2 = cross_val_score(best_model, X_scaled, y, cv=5, scoring='r2')
    r2_cv = cv_results_r2.mean()
    adj_r2_cv = adjusted_r2(r2_cv, X_scaled.shape[0], X_scaled.shape[1])

    print("\nCross-validation results:")
    print(f'Mean Cross-Validation MSE: {mse_cv}')
    print(f'Mean Cross-Validation RMSE: {rmse_cv}')
    print(f'Mean Cross-Validation MAE: {mae_cv}')
    print(f'Mean Cross-Validation R²: {r2_cv}')
    print(f'Mean Cross-Validation Adjusted R²: {adj_r2_cv}')

In [6]:
# Select most correlated features
def get_most_correlated_features(X, y, threshold=0.5):
    data_combined = pd.concat([X, y], axis=1)
    corr_matrix = data_combined.corr()
    target_corr = corr_matrix['Power'].drop('Power')
    
    # Select features above the correlation threshold
    most_correlated_features = target_corr[abs(target_corr) > threshold].index.tolist()
    print(f"Most correlated features: {most_correlated_features}")
    
    return most_correlated_features

In [7]:
# Main execution
if __name__ == "__main__":
    filepath = 'Location1.csv'
    
    # Load and preprocess data
    X, y = load_and_preprocess_data(filepath)
    
    # Train and evaluate the model on all features
    train_and_evaluate_rf(X, y)
    
    # Find most correlated features
    most_correlated_features = get_most_correlated_features(X, y, threshold=0.5)
    
    # Train and evaluate the model on the most correlated features
    print("\nEvaluating model on the most correlated features...")
    X_most_corr = X[most_correlated_features]
    train_and_evaluate_rf(X_most_corr, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters found:  {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None}

Random Forest Model Evaluation:
Mean Squared Error (MSE): 0.01975885172667169
Root Mean Squared Error (RMSE): 0.14056618272782287
Mean Absolute Error (MAE): 0.10439040205479452
R² Score: 0.7608692213239359
Adjusted R² Score: 0.7605958977684711

Cross-validation results:
Mean Cross-Validation MSE: 0.02942923971780621
Mean Cross-Validation RMSE: 0.17154952555401082
Mean Cross-Validation MAE: 0.13200177442922376
Mean Cross-Validation R²: 0.6451660719585204
Mean Cross-Validation Adjusted R²: 0.6450850392955134
Most correlated features: ['windspeed_10m', 'windspeed_100m', 'windgusts_10m']

Evaluating model on the most correlated features...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters found:  {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 5}

Random Forest